In [1]:
# Setup Keras model and save
import numpy as np
import onnx
import tensorflow as tf
from tensorflow import keras

from onnxruntime import get_device, __version__ 
print("Device: ", get_device())

print("TensorFlow: ", tf.__version__)
print("Keras: ", keras.__version__)
print("ONNX: ", onnx.__version__)

Device:  CPU
TensorFlow:  2.2.0
Keras:  2.3.0-tf
ONNX:  1.7.0


Saving a model to an artiface includes:
    The model's architecture/configuration
    Model's weight values
    Model's compilation information (if compile() is called)
    Optimizer and its state which enables you to restart training

A model can be saved using SavedModel format or older Keras H5 format. SavedModel is default when using model.save()


In [2]:
# Using SavedModel format
def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_model")

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 2ms/step - loss: 0.1984
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model\assets
4/4 [==============================] - 0s 2ms/step - loss: 0.1974


In [5]:
import keras2onnx
import onnxruntime

# Load Keras model
from tensorflow.keras.applications.resnet50 import ResNet50
model = ResNet50(include_top=True, weights='imagenet')

# alternative load method
#from tensorflow.keras.models import load_model

102973440/102967424 [==============================] - 8s 0us/step


In [6]:
# Convert to ONNX model
onnx_model = keras2onnx.convert_keras(model, model.name)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 458 -> 127


In [8]:
# Runtime Prediction
content = onnx_model.SerializeToString()
sess = onnxruntime.InferenceSession(content)
x = x if isinstance(x, list) else [x]
feed = dict([(input.name, x[n]) for n, input in enumerate(sess.get_inputs())])
pred_onnx = sess.run(None, feed)

NameError: name 'x' is not defined